# MEG Research

### _Project Description:_
Section III (Neural generation of electromagnetic fields) contains the basic physics of the signal generation in MEG. Subsections III.C and III.E.2 are the most relevant ones for the project. Implement equation 34 and compare the corresponding magnetic field distribution to the distribution that you get by calculating B with a numerical Biot-Savart implementation. The main outcome of the project is to assess whether the approximation of eq. 34 deviates significantly from the Biot-Savart solution in case of our "phantom head" that contains the manufactured current triangle loops with a 5 mm base length. 

In [1]:
import sympy as sp
import numpy as np
import math as math
import scipy.constants as const
import scipy.integrate as integrate
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import units as u
import pandas as pd

## Calculating the Magnetic Field (Assumption Method)

Here, "I" is the current from the sink at $r_{1}$ to the source at $r_{2}$, "Q" referrs to the current dipole. Q is defined as:

$$
Q = I\, \times\, (r_{2} - r_{1})
$$


### Magnetic Field Determined Via Assumption Method
Using the assumption method presented by Ilmoniemi _et al._ (1985) and by Sarvas (1987), the magnetic field in rectangular coordinates can be calculated by:

$$
\mathbf{B(r)} = \frac{\mu_{0}}{4 \pi} \frac{F\mathbf{Q} \times \mathbf{r_{Q}} - (\mathbf{Q} \times \mathbf{r_{Q}} \cdot \mathbf{r}) \nabla F(\mathbf{r},\mathbf{r_{Q}})}{F(\mathbf{r},\mathbf{r_{Q}})^{2}}
$$

where
$$
F(\mathbf{r}, \mathbf{r_{Q}}) = a(ra + r^{2} - \mathbf{r_{Q}} \cdot \mathbf{r})
$$

and
$$
\nabla F(\mathbf{r}, \mathbf{r_{Q}}) = (r^{-1}a^{2} + a^{-1}\mathbf{a} \cdot \mathbf{r} + 2a +2r)\mathbf{r} - (a + 2r + a^{-1}\mathbf{a} \cdot \mathbf{r})\mathbf{r_{Q}}
$$

with $a=(r-r_{Q})$, $\mathbf{a} = |a|$, and $r=|\mathbf{r}|$.

In [2]:
# Vacuum magnetic permeability: "permeability of free space"
from astropy.constants import mu0

In [3]:
def F(r_vector, rQ_vector):
    a_vector = r_vector - rQ_vector
    a = np.linalg.norm(a_vector)
    r = np.linalg.norm(r_vector)
    F = a * ((r * a) + (r**2) - np.dot(r_vector, rQ_vector))
    return F

def delF(r_vector, rQ_vector):
    a_vector = r_vector - rQ_vector
    a = np.linalg.norm(a_vector)
    r = np.linalg.norm(r_vector)
    delF = (((a**2 / r) + (np.dot(a_vecor, r_vector) / a) + (2 * a) + (2 * r)) * r_vector) - ((a + (2 * r) + (np.dot(a_vector, r_vector)/a)) * rQ_vector)
    return delF

In [4]:
# Calculating the Magnetic Field

def B(r_vector, rQ_vector, Q):
    B_numerator = (F(r_vector, rQ_vector) * np.cross(Q, rQ_vector)) - ((np.dot(np.cross(Q, rQ_vector), r_vector))*delF(r_vector, rQ_vector))
    B_denominator = F(r_vector, rQ_vector)**2
    magnetic_field = (mu0 / (4 * np.pi)) * (B_numerator / B_denominator)
    return magnetic_field

## Calculating the Magnetic Field (Biot-Savart Law)

### Magnetic Field From Biot-Savart Law
The magnetic field of a steady line current is geven by the **Biot-Savart law**:

$$
\mathbf{B(r)} = \frac{\mu_{0}}{4\pi} \int \frac{\mathbf{I}  \times \hat{\mathcal{R}}}{\mathscr{R}^{2}} dl' = \frac{\mu_{0}}{4\pi} I \int \frac{d\mathbf{l}' \times \hat{\mathcal{R}}}{\mathscr{R} ^{2}}
$$

Note:
- Current = **I** (with direction)
- Current Magnitude = |**I**| $\rightarrow$ labeled as "I"
- Distance = $d\mathbf{l}'$

### Position Vector
Position Vector:
$$
\mathbf{r} = x\, \hat{\mathbf{x}} + y\, \hat{\mathbf{y}} + z\, \hat{\mathbf{z}}
$$

Position Vector Magnitude:
$$
r = \sqrt{x^{2} + y^{2} + z^{2}}
$$

Distance from the origin:
$$
\hat{\mathbf{r}} = \frac{\mathbf{r}}{r} = \frac{x\, \hat{\mathbf{x}} + y\, \hat{\mathbf{y}} + z\, \hat{\mathbf{z}}}{\sqrt{x^{2} + y^{2} + z^{2}}}
$$

### Displacement Vector

Infinitesimal Displacement Vector:
$$
d\mathbf{l} = dx\, \hat{\mathbf{x}} + dy\, \hat{\mathbf{y}} + dz\, \hat{\mathbf{z}}
$$

Infinitesimal Displacement Vector Magnitude:
$$
dl = \sqrt{dx\,^{2} + dy\,^{2} + dz\,^{2}}
$$


### Separation Vector

Separation Vector
$$
\mathbf{\mathcal{R}} = \mathbf{r} - \mathbf{r}'
$$

Separation Vector Magnitude
$$
\mathscr{R} = |\mathbf{r} - \mathbf{r}'|
$$

Unit Vector in Direction From $\mathbf{r}'$ to $\mathbf{r}$
$$
\hat{\mathcal{R}} = \frac{\mathcal{R}}{\mathscr{R}} = \frac{\mathbf{r} - \mathbf{r}'}{|\mathbf{r} - \mathbf{r}'|}
$$

#### In Cartesian Coordinates:
$$
\mathcal{R} = (x-x')\, \hat{\mathbf{x}} + (y-y')\, \hat{\mathbf{y}} + (z-z')\, \hat{\mathbf{z}}
$$

$$
\mathscr{R} = \sqrt{(x-x')\, \hat{\mathbf{x}} + (y-y')\, \hat{\mathbf{y}} + (z-z')\, \hat{\mathbf{z}}}
$$

$$
\hat{\mathcal{R}} = \frac{(x-x')\, \hat{\mathbf{x}} + (y-y')\, \hat{\mathbf{y}} + (z-z')\, \hat{\mathbf{z}}}{\sqrt{(x-x')\, \hat{\mathbf{x}} + (y-y')\, \hat{\mathbf{y}} + (z-z')\, \hat{\mathbf{z}}}}
$$

Definitions:
- Source Point: $\mathbf{r}'$
- Field Point: $\mathbf{r}$
- Separation Vector: $\mathbf{\mathcal{R}}$
- Separation Vector Magnitude: $\mathscr{R}$

In [5]:
# Here, r and r_prime are 3D vectors with x, y, and z components corresponding to r, r_prime[0, 1, 2]

def scriptR_vector(r, r_prime):
    sep_vector = (r[0]-r[0]) + (r[1]-r_prime[1]) + (r[2]-r_prime[2])
    sep_vector_mag = np.sqrt(sep_vector)
    return sep_vector/sep_vector_mag

def scriptR_magnitude(r, r_prime):
    magnitude = np.sqrt((r[0] - r_prime[0]) + (r[1] - r_prime[1]) + (r[2] - r_prime[2]))
    return magnitude

# I, r, r_prime, dl_prime are 3D vectors with x, y, and z components

def B_current_vector(I, r, r_prime, dl_prime):
    scriptR = scriptR_vector(r, r_prime)
    scriptR_mag = scriptR_magnitude(r, r_prime)
    int_function = np.cross(I, scriptR) / scriptR_mag^2
    B = (mu0/(4*np.pi)) * sp.integrate(int_function, dl_prime)
    #evaluate integral
    #B.subs().n()
    return B

def B_length_vector(I, r, r_prime, dl_prime):
    scriptR = scriptR_vector(r, r_prime)
    scriptR_mag = scriptR_magnitude(r, r_prime)
    int_function = np.cross(dl_prime, scriptR) / scriptR_mag^2
    I_mag = np.sqrt(I[0]^2 + I[1]^2 + I[2]^2)
    B = (mu0/(4*np.pi)) * I_mag * sp.integrate(int_function)
    return B
